# Módulo 6 - Aprendizaje profundo
## Clase 3: RNN

### Modelo 1

# Ejemplos 1)
Modelo de clasificación de preguntas y respuestas

En esta práctica trabajaremos con un modelo de clasificación de texto, en el cual se procesarán preguntas sobre temas médicos para predecir la respuesta más probable entre una serie de respuestas posibles.

Este dataset contiene preguntas y respuestas relacionadas con temas de salud pública y enfermedades infecciosas, incluyendo VIH, neumonía, MERS-CoV, H1N1, hepatitis y otros.

## Pasos de la práctica

1. **Carga y limpieza de datos**: Usaremos un dataset con preguntas y respuestas reales. Primero, descargaremos estos datos y los limpiaremos, eliminando caracteres especiales y pasando todo el texto a minúsculas para facilitar el procesamiento.

2. **Preprocesamiento**: Convertiremos el texto en secuencias de números. Para ello, crearemos un "vocabulario", es decir, un diccionario donde cada palabra del dataset tendrá un número asociado. Esta etapa es clave para que el modelo entienda el texto.

3. **Definición del modelo**: Crearemos un modelo simple de clasificación con una capa de **embeddings** que convierte palabras en vectores de características y una capa completamente conectada que predice la respuesta entre varias opciones.

4. **Entrenamiento del modelo**: Usaremos un optimizador y una función de pérdida para entrenar el modelo. Durante el entrenamiento, el modelo intentará minimizar el error y mejorar la precisión de sus predicciones.

5. **Evaluación del modelo**: Evaluaremos el modelo para verificar su rendimiento en el conjunto de prueba. Mediremos la precisión para observar qué tan bien puede el modelo responder preguntas nuevas.

6. **Predicción de nuevas respuestas**: Finalmente, podrás probar el modelo con tus propias preguntas para ver la respuesta que genera en base al entrenamiento previo.

## Hiperparámetros

Durante esta práctica, tendrás la oportunidad de experimentar con los siguientes parámetros para observar su impacto en el modelo:

- **max_len**: Longitud máxima de las secuencias de palabras. Define cuántas palabras considera el modelo en cada pregunta.
- **embed_size**: Tamaño del vector de embedding, que representa cada palabra en un espacio numérico.
- **batch_size**: Número de ejemplos que procesa el modelo en cada iteración.
- **epochs**: Número de veces que el modelo recorre el dataset durante el entrenamiento.
- **learning_rate**: Tasa de aprendizaje que determina qué tan grandes son los ajustes que el modelo realiza en cada paso de entrenamiento.

## Procesos
- **Vocabulario**: Transformamos texto en números únicos para que el modelo comprenda las palabras.
- **Conversión**: Cambiamos cada palabra en una secuencia de índices, que el modelo puede procesar.
- **Truncamiento**: Controlamos la longitud de las preguntas para que todas sean iguales.
- **Padding**: Rellenamos con ceros las preguntas cortas para asegurar un tamaño constante de entrada.

In [1]:
import gdown
#https://github.com/deepset-ai/COVID-QA/tree/master/data/question-answering

print("Descargando dataset...")
url = 'https://drive.google.com/uc?export=download&id=1VBrfB70BATFUxtT1LZ-Z0Aq4Q_Ff9ZP0'
destination = "predictions.tsv"
gdown.download(url, destination, quiet=False)


Descargando dataset...


Downloading...
From: https://drive.google.com/uc?export=download&id=1VBrfB70BATFUxtT1LZ-Z0Aq4Q_Ff9ZP0
To: /content/predictions.tsv
100%|██████████| 77.7k/77.7k [00:00<00:00, 24.6MB/s]


'predictions.tsv'

In [6]:
import pandas as pd

# Cargar el dataset con pandas
data = pd.read_csv(destination, sep='\t')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question     249 non-null    object
 1   real_answer  249 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB
None


# Ejemplo con un modelo simple


Le mostramos un modelo inicial para que comprendan el flujo y la lógica usando el dataset proporcionado, antes de pasar a una versión más avanzada, que deben realizar.

1. **Embeddings**: Representan cada palabra de la pregunta en un espacio vectorial, lo que facilita que el modelo aprenda relaciones entre palabras.
2. **LSTM**: Procesa la secuencia de palabras y retiene información importante a lo largo de la frase.
3. **Capa Fully Connected**: Finalmente, convierte la salida de la LSTM en una predicción de la respuesta.





In [13]:
# Importar librerías necesarias
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import re

# Hiperparámetros
embed_size = 32          # Tamaño del embedding
hidden_size = 32         # Tamaño de la capa oculta LSTM
batch_size = 8           # Tamaño de cada lote de entrenamiento
max_len = 10             # Longitud máxima de la secuencia de palabras
learning_rate = 0.001    # Tasa de aprendizaje
epochs = 10               # Número de épocas de entrenamiento

# Cargar los datos y preprocesar texto
data = pd.read_csv("predictions.tsv", sep='\t')
data['question_clean'] = data['question'].apply(lambda x: re.sub(r"[^a-z0-9\s]", '', x.lower()))
data['answer_clean'] = data['real_answer'].apply(lambda x: re.sub(r"[^a-z0-9\s]", '', x.lower()))

# Codificar respuestas y texto
le = LabelEncoder()
data['label'] = le.fit_transform(data['answer_clean'])
vocab = {w: i+1 for i, w in enumerate(set(' '.join(data['question_clean']).split()))}
vocab['<pad>'] = 0
#["hola cómo estás"] -> [1, 2, 3] max_len = 3
#["bien"] -> [4] -> [1, 0, 0] "bien" + padding

def convertir_a_int(text):
    return [vocab.get(word, 0) for word in text.split()]

# Convertir y rellenar preguntas con padding
X = [convertir_a_int(text)[:max_len] + [0] * (max_len - len(text.split()[:max_len])) for text in data['question_clean']]
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#print(X)
#print(y)
#print(vocab)

# Convertir a tensores
X_train, X_test = torch.tensor(X_train), torch.tensor(X_test)
y_train, y_test = torch.tensor(y_train), torch.tensor(y_test)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size)

# Definir un modelo de clasificación simple con LSTM
class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(SimpleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# Instanciar el modelo y la función de pérdida
model = SimpleLSTM(len(vocab), embed_size, hidden_size, len(le.classes_))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Entrenar el modelo
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Época {epoch+1}, Pérdida: {loss.item():.4f}")

# Función de predicción
def predecir_respuesta(model, pregunta):
    model.eval()
    pregunta_int = convertir_a_int(re.sub(r"[^a-z0-9\s]", '', pregunta.lower()))
    pregunta_pad = torch.tensor([pregunta_int[:max_len] + [0] * (max_len - len(pregunta_int[:max_len]))])
    with torch.no_grad():
        output = model(pregunta_pad)
        pred = output.argmax(dim=1).item()
    return le.inverse_transform([pred])[0]

# Probar el modelo con una pregunta nueva
pregunta_nueva = "¿Cuál es la causa principal de la transmisión del VIH?"
print("Respuesta predicha:", predecir_respuesta(model, pregunta_nueva))

#Pregunta original: "¿Cuál es la causa principal de transmisión?"
#Secuencia numérica: [12, 23, 45, 34, 89]  # Índices de palabras
#Secuencia con padding: [12, 23, 45, 34, 89, 0, 0, 0, 0, 0]  # Si max_len = 10
#Respuestas únicas:
#["La transmisión es aérea", "Por contacto directo", "Uso de objetos contaminados"]
#[0, 1, 3]

Época 1, Pérdida: 4.3223
Época 2, Pérdida: 4.2087
Época 3, Pérdida: 4.3021
Época 4, Pérdida: 4.1995
Época 5, Pérdida: 4.1395
Época 6, Pérdida: 3.8889
Época 7, Pérdida: 3.7318
Época 8, Pérdida: 3.0722
Época 9, Pérdida: 2.8887
Época 10, Pérdida: 2.5504
Respuesta predicha: se ha introducido una definicin de caso revisada de neumona bacteriana presumida y esta definicin incluye casos de neumona con consolidacin alveolar definida por la oms as como aquellos con otras infiltraciones radiogrficas torcicas anormales y una protena c reactiva srica de al menos 40 mgl


In [ ]:
# Probar el modelo con una pregunta nueva
#pregunta_nueva = "¿ Qué virus de la influenza se identificó en China en 2013?" # bien
pregunta_nueva = "virus de la influenza se identificó en China en 2013?" # mal
#pregunta_nueva = "¿ Qué  virus de la influenza se descubrio en China en 2013?" # bien
#pregunta_nueva = "¿ Qué  virus de la influenza se descubrio en 2013?" # bien
#pregunta_nueva = "tratamiento para MERS-COV?"
respuesta_predicha = predecir_respuesta(model, pregunta_nueva)
print("\nPregunta:", pregunta_nueva)
print("Respuesta predicha:", respuesta_predicha)


Pregunta: virus de la influenza se identificó en China en 2013?
Respuesta predicha: h7n9


# Actividad

# TODO contruir un modelo LSTM

Deben construir su propio modelo basado en una LSTM (Long Short-Term Memory) para clasificación de texto. Este modelo será muy similar a los ejemplos que han visto en el cuaderno `RNN(refranes_and_poemas).ipynb`, donde trabajaron con RNNs para generación de texto, y en la celda anterior, donde vimos un ejemplo simple de modelo para clasificación con  el dataset actual.

**Objetivo**: Diseñar un modelo que pueda clasificar las preguntas en base a sus respuestas, utilizando una arquitectura LSTM. Este modelo tomará una secuencia de palabras y aprenderá a predecir la categoría o clase correspondiente a la respuesta.

### Recomendaciones y Pautas

Basado en el codigo anterior agregar lo siguiente:

1. **Agregar capa de embeddings.**  
2. **Implementar función de `accuracy`.**  
3. **Mejorar limpieza con regex.**  
4. **Incluir función `evaluate`.**  
5. **Añadir Dropout a la LSTM.**  
6. **Manejar estado oculto con `init_hidden`.**  
7. **Usar `LabelEncoder` para etiquetas.**  
8. **Incluir activación personalizada.**  
9. **Permitir múltiples capas LSTM (`num_layers`).**  
10. **Mostrar precisión y pérdida por época.**

Suerte !!!


In [ ]:
# Importar librerías necesarias
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # (7)
from torch.utils.data import DataLoader, TensorDataset
import re

# Hiperparámetros
#...          # Tamaño del embedding (1)
hidden_size = #...           # Tamaño de la capa oculta LSTM
#...            # Número de capas LSTM (9)
batch_size = #...          # Tamaño de cada lote de entrenamiento
max_len = #...              # Longitud máxima de la secuencia de palabras
learning_rate = #...      # Tasa de aprendizaje
epochs = #...                # Número de épocas de entrenamiento

# Función para limpiar texto (3)
def limpiar_texto(texto):
    """Limpia el texto eliminando caracteres no deseados y convierte a minúsculas."""
    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúüñ0-9\s]", '', texto)
    return texto.strip()

# Cargar los datos y preprocesar texto
data = pd.read_csv("predictions.tsv", sep='\t')
data['question_clean'] = data['question'].apply(limpiar_texto)  # Usamos la función limpiar_texto (3)
data['answer_clean'] = #...

# Codificar respuestas y texto
le = LabelEncoder()  # Usar LabelEncoder para etiquetas (7)
data['label'] = le.fit_transform(data['answer_clean'])

# Crear vocabulario con padding
vocab = {w: i+1 for i, w in enumerate(set(' '.join(data['question_clean']).split()))}
vocab['<pad>'] = 0

def convertir_a_int(text):
    return [vocab.get(word, 0) for word in text.split()]

# Convertir y rellenar preguntas con padding
X = [
    convertir_a_int(text)[:max_len] + [0] * (max_len - len(convertir_a_int(text)[:max_len]))
    for text in data['question_clean']
]
y = data['label'].values

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = #...

# Convertir a tensores
X_train = torch.tensor(X_train)
X_test = #...
y_train = torch.tensor(y_train)
y_test = #...

# Crear DataLoaders
train_loader = DataLoader(
    TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True
)
test_loader = #...

# Definir un modelo de clasificación con LSTM y Dropout
class CustomizableLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes, num_layers=1, dropout=0.0, activation='tanh'):
        super(CustomizableLSTM, self).__init__()
        #...   = hidden_size
        self.num_layers = num_layers

        self.embedding = #...   # Capa de embeddings (1)

        # LSTM con múltiples capas y Dropout (5, 9)
        self.lstm = nn.LSTM(
            embed_size, hidden_size, num_layers=#...
        )

        self.fc = nn.Linear#...

        # Activación personalizada (8)
        if activation == 'tanh':
            #...

    def forward(self, x, h):
        x = #...
        out, h = self.lstm(x, h)
        out = self.activation(out[:, -1, :])  # Aplicamos la activación personalizada (8)
        out = self.fc(out)
        return out, h

    def init_hidden(self, batch_size):
        # Inicializar el estado oculto y de memoria (6)
        return (
            torch.zeros(self.num_layers, #...  ),
            #...
        )

# Instanciar el modelo y la función de pérdida
vocab_size = len(vocab)
num_classes = len(le.classes_)
model = CustomizableLSTM(vocab_size, embed_size, hidden_size, num_classes, num_layers=#... )
optimizer = #...
criterion = #...

# Función para calcular precisión (2)
def accuracy(outputs, labels):
    preds = outputs.argmax(dim=1)
    return (preds == labels).sum().item() / #...

# Función para evaluar el modelo (4)
def evaluate(model, loader):
    model.eval()
    total_accuracy = 0
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in loader:
            h = model.init_hidden(inputs.size(0))  # Manejar estado oculto (6)
            outputs, h = model(inputs, h)
            loss = #...
            total_loss += loss.item()
            total_accuracy += #...
    avg_loss = total_loss / len(loader)
    avg_accuracy = total_accuracy / len(loader)
    return avg_loss, #...

# Entrenar el modelo
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_accuracy = 0  # Precisión en cada época (2)
    for inputs, labels in train_loader:
        h = model.init_hidden(inputs.size(0))  # Manejar estado oculto (6)
        #...
        outputs, h = model(inputs, h)
        loss = criterion(outputs, labels)
        loss.#...
        #...
        total_loss += #...
        total_accuracy += #...
    avg_loss = total_loss / #...
    avg_accuracy = total_accuracy / #...
    print(f"Época {epoch+1}, Pérdida de entrenamiento: {avg_loss:.4f}, Precisión de entrenamiento: {avg_accuracy:.4f}")  # Mostrar precisión y pérdida por época (10)

    # Evaluar después de cada época (4, 10)
    val_loss, val_accuracy =#...
    print(f"Precisión en validación: {val_accuracy:.4f}, Pérdida en validación: {val_loss:.4f}")

# Función de predicción
def predecir_respuesta(model, pregunta):
    model.#...
    pregunta_clean = limpiar_texto(pregunta)
    pregunta_int = convertir_a_int(pregunta_clean)
    pregunta_pad = [
        pregunta_int[:max_len] + [0] * (max_len - len(pregunta_int[:max_len]))
    ]
    pregunta_tensor = torch.tensor(pregunta_pad)
    h = model.init_hidden(1)  # Manejar estado oculto (6)
    with #... :
        output, h = #...
        pred = output.argmax(dim=1).item()
    return le.inverse_transform([pred])[0]

# Probar el modelo con una pregunta nueva
pregunta_nueva = "¿Cuál es la causa principal de la transmisión del VIH?"
print("Respuesta predicha:", predecir_respuesta(model, pregunta_nueva))


Época 1, Pérdida de entrenamiento: 4.4192, Precisión de entrenamiento: 0.0200
Precisión en validación: 0.0000, Pérdida en validación: 4.4428
Época 2, Pérdida de entrenamiento: 4.3782, Precisión de entrenamiento: 0.0450
Precisión en validación: 0.0000, Pérdida en validación: 4.4355
Época 3, Pérdida de entrenamiento: 4.2869, Precisión de entrenamiento: 0.1414
Precisión en validación: 0.0179, Pérdida en validación: 4.4054
Época 4, Pérdida de entrenamiento: 3.9588, Precisión de entrenamiento: 0.2079
Precisión en validación: 0.0179, Pérdida en validación: 4.2390
Época 5, Pérdida de entrenamiento: 3.3693, Precisión de entrenamiento: 0.4064
Precisión en validación: 0.1964, Pérdida en validación: 3.8358
Época 6, Pérdida de entrenamiento: 2.8666, Precisión de entrenamiento: 0.5786
Precisión en validación: 0.2679, Pérdida en validación: 3.4721
Época 7, Pérdida de entrenamiento: 2.4223, Precisión de entrenamiento: 0.7386
Precisión en validación: 0.3393, Pérdida en validación: 3.1779
Época 8, Pérd

In [ ]:
# Probar el modelo con una pregunta nueva
pregunta_nueva = "¿ Qué virus de la influenza se identificó en China en 2013?" # bien
pregunta_nueva = "¿ Qué  virus de la influenza se descubrio en China en 2013?" # bien
pregunta_nueva = "virus de la influenza en China en 2013?" #
pregunta_nueva = "¿Cuál es el tratamiento para MERS-COV?"
respuesta_predicha = predecir_respuesta(model, pregunta_nueva)
print("\nPregunta:", pregunta_nueva)
print("Respuesta predicha:", respuesta_predicha)


Pregunta: ¿Cuál es el tratamiento para MERS-COV?
Respuesta predicha: no hay un tratamiento específico para merscov al igual que la mayoría de las infecciones virales las opciones de tratamiento son de apoyo y sintomáticas


In [ ]:
# Una vez termiando el modelo anterior realicen lo siguinte:

# TODO 1: Modificar el número de épocas (epochs).
# Observa si al aumentar las épocas, la precisión en el conjunto de prueba sigue mejorando
# o si llega un punto en el que el modelo ya no muestra mejoras.

# TODO 2: Implementa "early stopping".
# Define cuántas épocas sin mejora en la pérdida quieres permitir antes de detener el entrenamiento.
# ¿Notas alguna diferencia en el tiempo de entrenamiento y la precisión final?

# TODO 3: Ajusta la longitud de las secuencias (max_len).
# ¿Produce una longitud de secuencia más larga o más corta un modelo que predice mejor las respuestas?

# TODO 4: Cambia la función de activación en el modelo LSTM.
# Prueba entre 'relu', 'tanh' e 'identity' y observa cuál captura mejor el estilo de respuesta.
# ¿A qué se puede deber la diferencia entre las activaciones?

# TODO 5: Reduce el tamaño de hidden_size.
# Observa si una capa oculta más pequeña sigue generando respuestas coherentes.
# ¿En qué afecta a la precisión del modelo?

# TODO 6: Añade una capa de dropout en el modelo.
# ¿Cómo impacta el dropout en la precisión y generalización? Observa si reduce la repetición en respuestas.

# TODO 7: Cambia la frase inicial para predicciones (pregunta_nueva).
# ¿El cambio en la frase inicial afecta el estilo de las respuestas?

# TODO 8: Ajusta el batch_size.
# Observa cómo impacta el tamaño del batch en la velocidad de entrenamiento y la precisión.

# TODO 9: Cambia la longitud de la respuesta generada (longitud en predecir_respuesta).
# ¿El modelo puede mantener coherencia en respuestas largas?

# TODO 10: Agrega gráficos de pérdida por época.
# Visualiza la pérdida para observar tendencias y detectar posibles problemas en el entrenamiento.
